In [1]:
import pandas as pd
import numpy as np
import env
import sklearn.preprocessing
from sklearn.model_selection import train_test_split

In [2]:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'


In [3]:
def acquire_zillow():
    '''
    Grab our data from path and read as csv
    '''
    
    df = pd.read_sql('SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips FROM properties_2017 join predictions_2017 using(parcelid) where propertylandusetypeid = 261;', get_connection('zillow'))

    return(df)

In [4]:
df = acquire_zillow()

In [5]:
df

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,4.0,3.5,3100.0,1023282.0,1998.0,11013.72,6059.0
1,2.0,1.0,1465.0,464000.0,1967.0,5672.48,6111.0
2,3.0,2.0,1243.0,564778.0,1962.0,6488.30,6059.0
3,4.0,3.0,2376.0,145143.0,1970.0,1777.51,6037.0
4,4.0,3.0,2962.0,773303.0,1950.0,9516.26,6037.0
...,...,...,...,...,...,...,...
52437,2.0,2.0,1286.0,354621.0,1940.0,4478.43,6037.0
52438,4.0,2.0,1612.0,67205.0,1964.0,1107.48,6111.0
52439,3.0,1.0,1032.0,49546.0,1954.0,876.43,6037.0
52440,3.0,2.0,1762.0,522000.0,1955.0,6317.15,6037.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52442 entries, 0 to 52441
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bedroomcnt                    52442 non-null  float64
 1   bathroomcnt                   52442 non-null  float64
 2   calculatedfinishedsquarefeet  52360 non-null  float64
 3   taxvaluedollarcnt             52441 non-null  float64
 4   yearbuilt                     52326 non-null  float64
 5   taxamount                     52438 non-null  float64
 6   fips                          52442 non-null  float64
dtypes: float64(7)
memory usage: 2.8 MB


In [7]:
def clean_zillow(df):
    '''
    Takes in a df of zillow_data and cleans the data 
    appropriatly by dropping nulls.

    return: df, a cleaned pandas data frame.
    '''
    
    # Instead of using dummies to seperate contracts use, 
    # df[['Contract']].value_counts()
    # Use a SQL querry
    
    df = df
    df = df.dropna()
    return df
    

In [9]:
df = clean_zillow(df)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52316 entries, 0 to 52441
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bedroomcnt                    52316 non-null  float64
 1   bathroomcnt                   52316 non-null  float64
 2   calculatedfinishedsquarefeet  52316 non-null  float64
 3   taxvaluedollarcnt             52316 non-null  float64
 4   yearbuilt                     52316 non-null  float64
 5   taxamount                     52316 non-null  float64
 6   fips                          52316 non-null  float64
dtypes: float64(7)
memory usage: 3.2 MB
